[문제] Managing Conversation History  
- trim_messages()
- RunnablePassthrough
- itemgetter()

세션ID 설정하여, 대화 진행합니다.(multi-turn conversation)
LLM 모델이 과거대화를 알지(기억) 못하는 상황을 만드세요.

[질문]
-육아휴직 기간은?  
-육아휴직 조건은?  
-육아휴직 대상자는?   
-육아휴직 수당 얼마?  
-첫 번째 질문이 뭐야?  
-질문 정리해줘.  
-네 역할은?  
-육아 휴직 질문에 대한 답변 정리해줘.  

In [53]:
## 1. 모듈(파일, 라이브러리) 읽어오기
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.chat_history import BaseChatMessageHistory

from dotenv import load_dotenv

## 2. 환경변수에 설정된 값 읽어오기
load_dotenv()

## 3. 파일읽기
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()

## 4. prompt 생성
template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
     ('placeholder', '{chat_history}'),
    ('user','{query}')
]

prompt = ChatPromptTemplate.from_messages(template)


In [56]:


## 5. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    # model='gpt-4.1-nano',
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

##########################################################################################
from langchain_core.messages import trim_messages

## 6.trim 설정
trimmer = trim_messages(
    max_tokens=65,        ## 최대 토큰 수 제한
    strategy='last',      ## 최근 메시지부터 시작해서 자르기
    token_counter=llm,    ## 모델 기준으로 토큰 수 계산
    include_system=True,  ## 시스템 메시지 포함
    allow_partial=False,  ## 메시지 자르기 제한
    start_on='human',      ## human 메시지부터 자르기 시작
)

from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

## 6. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
    | prompt 
    | llm
)
#############################################################################################


## 7. 세션별 대화 이력(히스토리)을 저장할 임시 메모리 저장소
store = {}

In [ ]:


## 8. 함수 정의 : 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


## 10. chain에 대화 이력(히스토리) 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)
## 11. 실행
while True:
    query = input('육아 휴직과 관련된 질문하세요 (종료하려면 S입력)')

    if query.upper() == 'S':
        break

    with_message_history.invoke(
        {'law': law, 'query': query},
        config={'configurable': {'session_id': 'abc'}}
    )
    print('\n'+'*' * 100 +'\n')


In [ ]:
store

{'abc': InMemoryChatMessageHistory(messages=[HumanMessage(content='육아휴직기간은?', additional_kwargs={}, response_metadata={}), AIMessage(content='육아휴직의 기간은 법령에 따라 최대 1년(12개월)입니다. 다만, 다음과 같은 경우에는 6개월(180일)까지 추가로 육아휴직을 사용할 수 있습니다:\n\n- 같은 자녀를 대상으로 부모가 각각 3개월 이상 육아휴직을 한 경우\n- 「한부모가족지원법」 제4조제1호의 부 또는 모\n- 고용노동부령으로 정하는 장애아동의 부 또는 모 (제19조(육아휴직) 제2항 참조)\n\n이 내용은 제19조(육아휴직) 제2항에 명확히 규정되어 있습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f'}, id='run-9035a3b5-d837-4729-80da-890bd47e6035-0')])}

In [ ]:
store.keys()

dict_keys(['abc'])

In [ ]:
for message in get_history('abc').messages:
    print(f'[{message.type.upper()}]:{message.content}\n')
    

[HUMAN]:육아휴직기간은?

[AI]:육아휴직의 기간은 법령에 따라 최대 1년(12개월)입니다. 다만, 다음과 같은 경우에는 6개월(180일)까지 추가로 육아휴직을 사용할 수 있습니다:

- 같은 자녀를 대상으로 부모가 각각 3개월 이상 육아휴직을 한 경우
- 「한부모가족지원법」 제4조제1호의 부 또는 모
- 고용노동부령으로 정하는 장애아동의 부 또는 모 (제19조(육아휴직) 제2항 참조)

이 내용은 제19조(육아휴직) 제2항에 명확히 규정되어 있습니다.

